In [1]:
#System and standard inputs
import sys
import importlib
import timeit
import collections

#Adding correct paths to custom modules
sys.path.append('/Users/sasrey/AlphaTrade/gym_exchange')
sys.path.append('/Users/sasrey/AlphaTrade')
sys.path.append('/Users/sasrey/AlphaTrade/gymnax_exchange')

In [2]:
#Relevant Jax Imports
import jax.numpy as jnp
from jax import lax
import jax

In [134]:
#Direct Import of the Orderbook functions 
import jaxob.JaxOrderbook as job 
#OrderBook class import as written by Kang
from orderbook import OrderBook as cpuOrderBook
#Import (and reload for debugging) of the OrderBook class with jax backend (SF) should behave in the same way.
import jorderbook
importlib.reload(jorderbook)
from jorderbook import OrderBook as JaxOrderBook

#Import Data Mgt and Encoding/Decoding tools
#(The aim of these is to adjust for the case where no message is received but data ends up in the L2 data)
import gym_exchange.data_orderbook_adapter.data_pipeline as dpl
from gym_exchange.data_orderbook_adapter.decoder import Decoder
from gym_exchange.data_orderbook_adapter.encoder import Encoder

In [ ]:
#Attempt at using the data pipeline tools and the encoder/decoder, doesn't really work well. 

dplOBJ=dpl.DataPipeline()
decodeddata=Decoder(10,40,dplOBJ.historical_data,dplOBJ.data_loader)
#Decoder Functionality: Takes the message data (dataloader) and the orderbook data (historical data).
#Creates and maintains an orderbook as implemented in the CPU version.
#Creates and maintains a "DataAdjuster" object, which considers the historical data (orderbook) to be the "true" state
#Inits the orderbook with the first state in the orderbook data (hist) gives imaginary list of IDs to the orders created:
#            each entry in the orderbook is considered to be from a single order. 
output=decodeddata.process()
#Process function for Decoder: goes through N (horizon) messages to ascertain whether there are adidtional messages to consider. 

enc=Encoder(decodeddata)
#enc()

In [245]:
import pandas as pd
df_msg = pd.read_csv("/Users/sasrey/AlphaTrade/data/TSLA_2015-01-02_34200000_57600000_message_50.csv", header=None)
df_ob = pd.read_csv("/Users/sasrey/AlphaTrade/data/TSLA_2015-01-02_34200000_57600000_orderbook_50.csv", header=None)
orderbookLevels=50

In [248]:


data=jnp.array(df_ob.iloc[0,:]).reshape(int(orderbookLevels*2),2)
newarr=jnp.zeros((int(orderbookLevels*2),7))
initOB=newarr.at[:,3].set(data[:,0]).at[:,2].set(data[:,1]).at[:,0].set(1).at[0:orderbookLevels*4:2,1].set(-1).at[1:orderbookLevels*4:2,1].set(1).at[:,4].set(0).at[:,5].set(job.INITID).at[:,6].set(34200.00000000)


df_orders=df_msg.rename(columns={1:'Type',5:'Side',3:'Quantity',4:'Price',2:'OrderID',0:'Time'})
df_orders['TradeID']=0
col_order=['Type','Side','Quantity','Price','TradeID','OrderID','Time']
df_orders=df_orders[col_order]
df_orders_simple=df_orders[(df_orders['Type']!=6)&(df_orders['Type']!=7)&(df_orders['Type']!=5)]

ordersJNP=jnp.array(df_orders)
print(ordersJNP.shape)

(290737, 7)


In [253]:
orderbook=JaxOrderBook(price_levels=50)
trades_init=orderbook.process_orders_array(initOB)
trades_msgs=orderbook.process_orders_array(ordersJNP[0:200])

In [254]:
myOB=orderbook.get_L2_state()
myOB

Array([2.2291e+06, 1.5070e+03, 2.2260e+06, 2.0000e+02, 2.2300e+06,
       2.6000e+01, 2.2259e+06, 1.0000e+02, 2.2335e+06, 2.3000e+01,
       2.2251e+06, 2.8000e+01, 2.2340e+06, 5.0000e+01, 2.2250e+06,
       9.7900e+02, 2.2350e+06, 3.9000e+02, 2.2249e+06, 8.0000e+00,
       2.2368e+06, 1.0000e+02, 2.2245e+06, 1.7000e+01, 2.2375e+06,
       1.0000e+02, 2.2244e+06, 2.0000e+00, 2.2400e+06, 2.5000e+02,
       2.2241e+06, 2.6500e+02, 2.2410e+06, 1.0000e+02, 2.2240e+06,
       4.2000e+01, 2.2425e+06, 2.0000e+01, 2.2222e+06, 1.0000e+00,
       2.2475e+06, 2.0000e+01, 2.2221e+06, 1.0000e+01, 2.2481e+06,
       5.0000e+01, 2.2220e+06, 6.0000e+00, 2.2495e+06, 1.0300e+02,
       2.2217e+06, 1.5000e+01, 2.2500e+06, 5.7600e+02, 2.2215e+06,
       2.0000e+02, 2.2510e+06, 1.0000e+02, 2.2214e+06, 2.0000e+01,
       2.2524e+06, 4.0000e+02, 2.2210e+06, 1.4000e+01, 2.2525e+06,
       4.5000e+01, 2.2207e+06, 1.0000e+02, 2.2539e+06, 1.0000e+02,
       2.2200e+06, 7.6100e+02, 2.2545e+06, 1.0000e+01, 2.2198e

In [255]:
L2OB=jnp.array(df_ob.iloc[199]).astype(float)
L2OB

Array([2.2291e+06, 1.5070e+03, 2.2260e+06, 2.0000e+02, 2.2300e+06,
       2.6000e+01, 2.2259e+06, 1.0000e+02, 2.2335e+06, 2.3000e+01,
       2.2251e+06, 2.8000e+01, 2.2340e+06, 5.0000e+01, 2.2250e+06,
       9.7900e+02, 2.2350e+06, 3.9000e+02, 2.2249e+06, 8.0000e+00,
       2.2368e+06, 1.0000e+02, 2.2245e+06, 1.7000e+01, 2.2375e+06,
       1.0000e+02, 2.2244e+06, 2.0000e+00, 2.2400e+06, 2.5000e+02,
       2.2241e+06, 2.6500e+02, 2.2410e+06, 1.0000e+02, 2.2240e+06,
       4.2000e+01, 2.2425e+06, 2.0000e+01, 2.2222e+06, 1.0000e+00,
       2.2475e+06, 2.0000e+01, 2.2221e+06, 1.0000e+01, 2.2481e+06,
       5.0000e+01, 2.2220e+06, 6.0000e+00, 2.2495e+06, 1.0300e+02,
       2.2217e+06, 1.5000e+01, 2.2500e+06, 8.2400e+02, 2.2215e+06,
       2.0000e+02, 2.2510e+06, 1.0000e+02, 2.2214e+06, 2.0000e+01,
       2.2524e+06, 4.0000e+02, 2.2210e+06, 1.4000e+01, 2.2525e+06,
       4.5000e+01, 2.2207e+06, 1.0000e+02, 2.2539e+06, 1.0000e+02,
       2.2200e+06, 1.3810e+03, 2.2545e+06, 1.0000e+01, 2.2198e

In [256]:
L2OB-myOB

Array([ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  2.480e+02,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  6.200e+02,  0.000e+00,  0.000e+00,  0.000e